# Przetwarzanie języka naturalnego (ang. natural language processing, NLP)

**Autorzy:** S. Mandes, A. Karlińska
## Wprowadzanie 

### Co to jest NLP?

"Interdyscyplinarna dziedzina, łącząca zagadnienia sztucznej inteligencji i językoznawstwa, zajmująca się automatyzacją analizy, rozumienia, tłumaczenia i generowania języka naturalnego przez komputer." (źródło: wikipedia pl --> polecam cały wpis. A jeszcze lepiej wpis na eng wiki)

A mówiąc bardziej konkretnie:

NLP rozwija się jako odpowiedź na problemy: a) przetwarzania, b) klasyfikowania i c) analizy "treści" bardzo dużych zbiorów tekstów w formie elektronicznej. 

Ad. a) kiedy wpisujemy jakiś tekst w wyszukiwarkę, to odpowiedź wyświetla się błyskawicznie. Czyli nie jest tak, że po wpisaniu zapytania jakiś program przeszukuje dla nas odpowiedzi na pytanie. Ono już jest (zazwyczaj) gotowe. Zawartość internetu (pewna jego część, tak naprawdę) została przetworzona pod kątem potencjalnych pytań / wyszukiwań. Problem pierwszy to, jak przetwarzać tekst, biorąc pod uwagę, że komputer przetwarza liczby. Jak przetwarzać efektywnie? itd.

Ad. b) kiedy wyszukujemy jakiś produkt, to otrzymujemy od razu jakieś sugestie. To samo dotyczy filmów, książek, wszystkiego. Problem drugi polega na grupowaniu treści w większe całości (sieci znaczeń), które umożliwią "wnioskowanie". Inny klasyczny problem: jak rozpoznać, że coś jest spamem, a coś nie? 

Ad. c) jak rozpoznać, o czym jest mowa w tekście? Jaki jest emocjonalny wydźwięk tekstu? Jak rozpoznać treści pytania, które kierujemy do Alexy / Siri / itd. (chat bot)? Obecnie główny problem: jak rozpoznać fake newsy?

### A jak to ma się do socjologii?


### Materiały dodatkowe:

SpaCy:
https://spacy.io/usage/spacy-101#_title

Dlaczego język polski trudnym jest:

https://www.youtube.com/watch?v=WY6zrqF9q-w&t=1824s
(1 połowa - wystąpienie Ł. Kobylińskiego)

O NLP - trochę historii i co się dzieje współcześnie:

https://www.youtube.com/watch?v=IUbFMt_4_Hw

## Tematy zajęć:

- SpaCy - podstawy
- tokenizacja
- stemming
- lematyzacja
- stop words

___
# Praca z SpaCy w Pythonie

Jest to typowy zestaw instrukcji do importowania i pracy ze apaCy. To może zająć trochę czasu - spaCy posiada dość dużą bibliotekę do załadowania. 

`sm` przy modelu języka oznacza small version. To jest model, który będzie wykorzystany do analizy. Za jakiś czas użyjemy pełnego modelu.

In [2]:
import spacy
nlp = spacy.load('en_core_web_sm')

In [61]:
# tworzymy Doc object i przeprowadzamy tokenizację. 
doc = nlp('Facebook is going to buy U.S. startup for $6 million')


In [62]:
type(doc)

spacy.tokens.doc.Doc

In [63]:
'''Wyświetlamy każdy token odddzielnie. pos - part of speech 
(bez _ pokaże cyfry, zakodowane częsci mowy, # dep - syntactic dependency'''

for token in doc:
    print(token.text, token.pos_, token.dep_)

Facebook PROPN nsubj
is AUX aux
going VERB ROOT
to PART aux
buy VERB xcomp
U.S. PROPN compound
startup NOUN dobj
for ADP prep
$ SYM quantmod
6 NUM compound
million NUM pobj


Nie wygląda to zbyt jasno, ale od razu widzimy, że dzieje się kilka ciekawych rzeczy:

1. Facebook został rozpoznany jako nazwa własna, a nie tylko słowo na początku zdania
2. Stany Zjednoczone zostały rozpoznane jako nazwa pomimo kropek (nazywamy to "tokenem")

Później wyjaśnimy pozostałe skróty.


___
# Pipeline
Kiedy uruchamiamy `nlp`, tekst przechodzi przez *processing pipeline*, który najpierw rozbija tekst na podstawowe elementy, a następnie wykonuje serię operacji, aby oznaczyć, przetworzyć i opisać dane. Co dokładnie robi?

https://spacy.io/pipeline-7a14d4edd18f3edfee8f34393bff2992.svg

Możemy sprawdzić, jakie elementy znajdują się obecnie w pipeline.
Ważne, bo można sobie różne procesy do pipline wstawiać.

In [65]:
nlp.pipeline

[('tagger', <spacy.pipeline.pipes.Tagger at 0x27c980cc4c8>),
 ('parser', <spacy.pipeline.pipes.DependencyParser at 0x27c980b0ee8>),
 ('ner', <spacy.pipeline.pipes.EntityRecognizer at 0x27c980cd4c8>)]

## Tokenizacja
Pierwszym krokiem w przetwarzaniu tekstu jest podzielenie wszystkich części składowych (słów i interpunkcji) na "tokeny". Są one opatrzone adnotacjami wewnątrz obiektu Doc, które zawierają więcej informacji o poszczególnych tokenach. Więcej szczegółów na temat tokenizacji za chwilę. Na razie spójrzmy na inny przykład:

In [2]:
doc2 = nlp("Facebook isn't going to buy U.S. startup for $6 million anymore.")

for token in doc2:
    print(token.text, token.pos_, token.dep_)

Facebook PROPN nsubj
is AUX aux
n't PART neg
going VERB ROOT
to PART aux
buy VERB xcomp
       SPACE 
U.S. PROPN compound
startup NOUN dobj
for ADP prep
$ SYM quantmod
6 NUM compound
million NUM pobj
anymore ADV advmod
. PUNCT punct


SpaCy rozpoznaje zarówno sam czasownik, jak i dołączoną do niego negację. 

Uwaga: zarówno rozszerzona "biała przestrzeń" (whitespace) jak i kropka na końcu zdania są przypisane do własnych tokenów.

Należy zauważyć, że nawet jeśli doc2 zawiera przetworzone informacje o każdym tokenie, zachowuje on również oryginalny tekst:

In [15]:
doc2

Facebook isn't going to buy       U.S. startup for $6 million anymore.

In [66]:
doc2[0]

Facebook

___
## Part-of-Speech Tagging (POS)

Następnym krokiem po podzieleniu tekstu na tokeny jest przypisanie informacji o części mowy. W powyższym przykładzie `Facebook` został uznany za ***nazwę własną***. Tutaj wymagane jest pewne statystyczne modelowanie. Na przykład,słowa, które następują po "the", są zazwyczaj rzeczownikami.

Więcej: https://spacy.io/api/annotation#pos-tagging

In [3]:
doc2[0].pos_

'PROPN'

___
## Dependencies 

SpaCy określa również zależności składniowe tokenów. `Facebook` jest identyfikowany jako `nsubj` lub ***nominal subject*** zdania.

więcej: https://spacy.io/api/annotation#dependency-parsing

In [4]:
doc2[0].dep_

'nsubj'

In [5]:
spacy.explain('PROPN')

'proper noun'

In [6]:
spacy.explain('nsubj')

'nominal subject'

## Co jeszcze "ukryte" jest w tokenach? 

In [9]:
print(doc2[3].text)

going


In [8]:
# lemat (podstawowa forma słowa):

print(doc2[3].lemma_)

going
go


In [17]:
# informacja o o tym, jaka to jest część zdania: ogólna i dokładna
print(doc2[3].pos_)
print(doc2[3].tag_ + ' / ' + spacy.explain(doc2[3].tag_))

VERB
VBG / verb, gerund or present participle


In [15]:
# Word Shapes:
print(doc2[0].text+': '+doc2[0].shape_)
print(doc[5].text+' : '+doc[5].shape_)

Facebook: Xxxxx
U.S. : X.X.


In [16]:
# Boolean Values:
print(doc2[0].is_alpha)
print(doc2[0].is_stop)

True
False


___
## Spans

Duże teksty mogą być czasami trudne do obróbki. **span** służy do dzielenia tekstu.


In [30]:
doc3 = nlp("""Natural language processing (NLP) is a subfield of linguistics, computer science, 
information engineering, and artificial intelligence concerned with the interactions between 
computers and human (natural) languages, in particular how to program computers to process and
analyze large amounts of natural language data. The history of natural language processing (NLP) 
generally started in the 1950s, although work can be found from earlier periods. In 1950, Alan Turing 
published an article titled "Computing Machinery and Intelligence" which proposed what is now called 
the Turing test as a criterion of intelligence.""")

In [36]:
frag = doc3[10:30]
print(frag)

linguistics, computer science, 
information engineering, and artificial intelligence concerned with the interactions between 
computers and


In [37]:
type(frag)


spacy.tokens.span.Span

___
## Zdania
Niektóre tokeny wewnątrz tekstu mogą również otrzymać znacznik "początek zdania". Chociaż nie buduje to natychmiast listy zdań, tagi te umożliwiają generowanie segmentów zdań poprzez `Doc.sents`.


In [56]:
for sent in doc3.sents:
    print(sent, "\n")


Natural language processing (NLP) is a subfield of linguistics, computer science, 
information engineering, and artificial intelligence concerned with the interactions between 
computers and human (natural) languages, in particular how to program computers to process and
analyze large amounts of natural language data. 

The history of natural language processing (NLP) 
generally started in the 1950s, although work can be found from earlier periods. 

In 1950, Alan Turing 
published an article titled "Computing Machinery and Intelligence" which proposed what is now called 
the Turing test as a criterion of intelligence. 



In [60]:
doc3[0].is_sent_start

True

---
## Tokenizacja

In [5]:
# nieco bardziej skomplikowane przypadki stringów
txt = '"Let\'s go to N.Y.!"'
print(txt)

"Let's go to N.Y.!"


In [6]:
doc = nlp(txt)

In [7]:
for token in doc:
    print(token.text, end=' | ')

" | Let | 's | go | to | N.Y. | ! | " | 

Co się dzieje pod spodem?

https://spacy.io/usage/spacy-101#annotations-token

### Prefixes, Suffixes, Infixes
SpaCy odizoluje interpunkcję, która nie *nie* stanowi integralnej części słowa. Cudzysłowy, przecinki i interpunkcja na końcu zdania będą oddzielnymi tokenami. 

Ale nie zawsze. Interpunkcja, która istnieje jako część adresu e-mail, strony internetowej lub wartości numerycznej, będzie zachowana jako część tokenu.

- Prefix: Character(s) at the beginning, e.g. $, (, “, ¿.
- Suffix: Character(s) at the end, e.g. km, ), ”, !.
- Infix: Character(s) in between, e.g. -, --, /, ….
- Exception: Special-case rule to split a string into several tokens or prevent a token from being split when punctuation rules are applied, e.g. `St. U.S.`

In [8]:
doc2 = nlp("Help us! Send an e-mail support@website.com or visit us at http://www.website.com!")

for t in doc2:
    print(t)

Help
us
!
Send
an
e
-
mail
support@website.com
or
visit
us
at
http://www.website.com
!


Uwaga: e-mail został podzielony na 3 tokeny. Czyli jak widać, nie zawsze to działa, jak byśmy chcieli lub myśleli, że powinno działać!

https://english.stackexchange.com/questions/1925/email-or-e-mail

In [9]:
# tokeny możemy policzyć
len(doc2)

15

In [12]:
# tokeny możemy przetwarzać po indeksach
doc2[2]

!

In [13]:
# ale nie można zmieniać
doc2[2] = doc[1]

TypeError: 'spacy.tokens.doc.Doc' object does not support item assignment

## Rozpoznawanie nazw własnych



In [18]:
doc = nlp('Facebook is going to buy Polish startup for $6 million from Warsaw')

In [19]:
# wydobywamy z tekstu nazwy własne. Ale też 6 milionów
# uwaga, do nazwy dokumentu dodajemy .ents

for ent in doc.ents:
    print(ent)

Polish
$6 million
Warsaw


In [20]:
# możemy od razu je zaklasyfikować (label, znowu z _)

for ent in doc.ents:
    print(ent.text+' - '+ent.label_+' - '+str(spacy.explain(ent.label_)))

Polish - NORP - Nationalities or religious or political groups
$6 million - MONEY - Monetary values, including unit
Warsaw - GPE - Countries, cities, states


In [21]:
len(doc.ents)

3

## Noun chunks (kawałki rzeczownika??)
czyli rzeczowniki z słowami opisujacymi / określającymi je.

- Text: The original noun chunk text.
- Root text: The original text of the word connecting the noun chunk to the rest of the parse.
- Root dep: Dependency relation connecting the root to its head.

https://spacy.io/usage/linguistic-features#noun-chunks

In [23]:
doc3 = nlp("Autonomous cars shift insurance liability toward manufacturers")
for chunk in doc3.noun_chunks:
    print(chunk.text)

Autonomous cars
insurance liability
manufacturers


In [31]:
doc3 = nlp("Autonomous cars shift insurance liability toward manufacturers")
for chunk in doc3.noun_chunks:
    print(f'{chunk.text:{22}} {chunk.root.text:{21}} {chunk.root.dep_}')

Autonomous cars        cars                  nsubj
insurance liability    liability             dobj
manufacturers          manufacturers         pobj


___
# Built-in Visualizers

spaCy posiada narzędzie do wizualizacji relacji pomiędzy tokenami **displaCy**. 

Uwaga: najlepiej działa w Jupyter. Można też robić w innych narzędziach, ale wymaga eksportu do HTML.


Więcej informacji: https://spacy.io/usage/visualizers

In [32]:
from spacy import displacy

In [33]:
doc = nlp('Over the last quarter Apple sold nearly 20 thousand iPods for a profit of $6 million.')
displacy.render(doc, style='ent', jupyter=True)


---
## Stemming

W wyszukiwaniu informacji oraz w morfologii (w językoznawstwie) jest to proces usunięcia ze słowa końcówki fleksyjnej, pozostawiając tylko temat wyrazu. Proces stemmingu może być przeprowadzany w celu zmierzenia popularności danego słowa. Końcówki fleksyjne zaniżają faktyczne dane. Algorytmy stemmingu są przedmiotem badań informatyki od lat 60. XX wieku. Pierwszy stemmer, czyli program do przeprowadzania procesu stemmingu, został napisany i opublikowany przez Julie Beth Lovins w 1968. W czerwcu 1980 Martin Porter opublikował swój algorytm stemmingu, zwany Algorytmem Portera.

Np. angielskie słowa: „connection”, „connections”, „connective”, „connected”, „connecting” poddane stemmingowi dadzą ten sam wynik, czyli słowo „connect”.

źródło: wikipedia

W uproszczeniu, stemming to obcięcie wszelkiego rodzaju przedrostków i przyrostków, w celu dotarcia do „rdzenia” wyrazu. Rdzeń nie musi być poprawnym słowem.

SpaCy nie posiada narzędzia do stemmingu --> por. dyskusja o zaletach stemmingu w porównaniu do lematyzacji:

https://github.com/explosion/spaCy/issues/327

In [39]:
# importujemy NLTK i całą biblitekę do stemmingu w wydaniu Portera
import nltk

from nltk.stem.porter import *

In [40]:
p_stemmer = PorterStemmer()

In [43]:
words1 = ['connection', 'connections', 'connective', 'connected', 'connecting']

In [46]:
for word in words1:
    print(word+' --> '+p_stemmer.stem(word))

connection --> connect
connections --> connect
connective --> connect
connected --> connect
connecting --> connect


In [45]:
# ale nie zawsze to działa dobrze, np problem z przysłówkami
words2 = ['easily', 'fairly', 'quitely', 'entirely']

In [47]:
for word in words2:
    print(word+' --> '+p_stemmer.stem(word))

easily --> easili
fairly --> fairli
quitely --> quit
entirely --> entir


Jest też druga wersja tzw. "Snowball Stemmer" zwany też "English Stemmer" or "Porter2 Stemmer"

In [48]:
from nltk.stem.snowball import SnowballStemmer

In [49]:
s_stemmer = SnowballStemmer(language='english')

In [50]:
for word in words1:
    print(word+' --> '+s_stemmer.stem(word))

connection --> connect
connections --> connect
connective --> connect
connected --> connect
connecting --> connect


In [51]:
for word in words2:
    print(word+' --> '+s_stemmer.stem(word))

easily --> easili
fairly --> fair
quitely --> quit
entirely --> entir



----
## Lematyzacja

Jest to proces sprowadzania zbioru słów do lematu (podstawowej postaci). W przypadku czasownika będzie do bezokolicznik, w przypadku rzeczownika – mianownik liczby pojedynczej. Do wykonania tego zadania potrzebny jest słownik lub rozbudowany zestaw reguł fleksyjnych dla danego języka.

**Lemat** - ta spośród form gramatycznych wyrazu odmiennego, która jest tradycyjnie wykorzystywana w słownikach i reprezentuje tam w nagłówku artykułu hasłowego cały wyraz ze wszystkimi jego formami. Forma ta stanowi niejako „umowną etykietę zbioru form” i decyduje o umiejscowieniu artykułu hasłowego w słowniku.

Wybór formy słownikowej bywa ustalony tradycją i różni się w zależności od języka.

Forma słownikowa bywa zwykle traktowana przez osoby uczące się języka obcego jako podstawowa, tj. taka, w której przyswajane są nowe wyrazy i od której tworzy się pozostałe formy odmiany.
                                                                  
                                                              źródło: wikipedia

In [72]:
doc4 = nlp("I am a biker biking in a race because I love to bike and I bike today")

for token in doc4:
    print(token.lemma, '\t\t', token.lemma_)

561228191312463089 		 -PRON-
10382539506755952630 		 be
11901859001352538922 		 a
12223578186674142820 		 biker
16029548483725639901 		 bike
3002984154512732771 		 in
11901859001352538922 		 a
8048469955494714898 		 race
16950148841647037698 		 because
561228191312463089 		 -PRON-
3702023516439754181 		 love
3791531372978436496 		 to
16029548483725639901 		 bike
2283656566040971221 		 and
561228191312463089 		 -PRON-
16029548483725639901 		 bike
11042482332948150395 		 today


In [73]:
for lem in doc4:
    print(f'{lem.text:{10}} {lem.pos_:{10}} {lem.lemma:{20}} {lem.lemma_:{10}}')

I          PRON         561228191312463089 -PRON-    
am         AUX        10382539506755952630 be        
a          DET        11901859001352538922 a         
biker      NOUN       12223578186674142820 biker     
biking     VERB       16029548483725639901 bike      
in         ADP         3002984154512732771 in        
a          DET        11901859001352538922 a         
race       NOUN        8048469955494714898 race      
because    SCONJ      16950148841647037698 because   
I          PRON         561228191312463089 -PRON-    
love       VERB        3702023516439754181 love      
to         PART        3791531372978436496 to        
bike       VERB       16029548483725639901 bike      
and        CCONJ       2283656566040971221 and       
I          PRON         561228191312463089 -PRON-    
bike       VERB       16029548483725639901 bike      
today      NOUN       11042482332948150395 today     


In [75]:
def pokaż_lematy(text):
    for token in text:
        print(f'{token.text:{10}} {token.pos_:{10}} {token.lemma:{20}} {token.lemma_:{10}}')

In [85]:
# a co się stanie, jak użyjemy nieistniejącego słowa 
pokaż_lematy(nlp('I want to show you something!'))

I          PRON         561228191312463089 -PRON-    
want       VERB        7597692042947428029 want      
to         PART        3791531372978436496 to        
show       VERB        1916734850589852068 show      
you        PRON         561228191312463089 -PRON-    
something  PRON       17370494668576369452 something 
!          PUNCT      17494803046312582752 !         


In [84]:
# mamy w poniższym zdaniu dwa razy meeting
doc5 = nlp("I am meeting him tomorrow at the meeting.")

pokaż_lematy(doc5)

I          PRON         561228191312463089 -PRON-    
am         AUX        10382539506755952630 be        
meeting    VERB        6880656908171229526 meet      
him        PRON         561228191312463089 -PRON-    
tomorrow   NOUN        3573583789758258062 tomorrow  
at         ADP        11667289587015813222 at        
the        DET         7425985699627899538 the       
meeting    NOUN       14798207169164081740 meeting   
.          PUNCT      12646065887601541794 .         
